In [ ]:
!pip install psycopg2-binary
!pip3 install pyTelegramBotAPI
!pip install schedule

In [3]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import schedule
import time

import telebot
import logging

Подтягиваем данные из развернутой локальной БД

In [ ]:
conn = psycopg2.connect(
    dbname="vitalijflerin",
    user="postgres",
    password="Cfgaj",
    host="localhost",
    port="5432"
)

In [ ]:
cur = conn.cursor()

In [54]:
cur.execute("SELECT * FROM orders")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
temp = df.loc[0, df.columns[1]]
temp

'CA-2018-152156'

работа с ботом

In [30]:
token = '5/////k'
your_chat_id = '3///'
chat_id = '3////'
bot = telebot.TeleBot(token)

# Добавляем обработчики команд
@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    if message.from_user.id != bot.get_me().id:
        bot.reply_to(message, "Привет! Чем тебе помочь?")

@bot.message_handler(commands=['hello'])
def send_hello(message):
    if message.from_user.id != bot.get_me().id:
        t = "Good day to you too!" # Заменили данные на сообщение приветствия
        t_str = str(t)
        # Send the message to the user
        bot.send_message(chat_id=message.chat.id, text=t_str)

@bot.message_handler(commands=['nikita', 'никита', 'Никита', 'Nikita'])
def send_hello(message):
    if message.from_user.id != bot.get_me().id:
        t1 = "Тебе уже ничем не помочь..." # Заменили данные на сообщение приветствия
        t_str1 = str(t1)
        # Send the message to the user
        bot.send_message(chat_id=message.chat.id, text=t_str1)

@bot.message_handler(commands=['inf'])
def send_hello(message):
    if message.from_user.id != bot.get_me().id:
        t2 = temp # Заменили данные на сообщение приветствия
        t_str2 = str(t2)
        # Send the message to the user
        bot.send_message(chat_id=message.chat.id, text=t_str2)


@bot.message_handler(commands=['plot'])
def send_plot(message):
    if message.from_user.id != bot.get_me().id:
        # Создаем данные для графика
        x = np.arange(1, 10, 1)
        y = np.random.choice(5, len(x))

        # Создаем график
        sns.lineplot(x, y)
        plt.title('test')

        # Сохраняем график в объект io.BytesIO
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)
        
        # Отправляем график через бота
        bot.send_photo(chat_id=message.chat.id, photo=plot_object)


df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': ['apples', 'oranges', 'bananas']})

# Добавляем обработчик команды для отправки датафрейма
@bot.message_handler(commands=['df'])
def send_dataframe(message):
    if message.from_user.id != bot.get_me().id:
        # Сериализуем датафрейм в строку
        df_csv = df.to_csv(index=False)

        # Кодируем строку в байты
        df_bytes = io.BytesIO(df_csv.encode())

        # Отправляем файл
        bot.send_document(chat_id=your_chat_id, document=df_bytes, caption='my_dataframe.csv')

# Запускаем процесс диспетчеризации
bot.polling()

In [ ]:
# отправлем сообщение всем подписчикам
#Получаем список всех подписчиков бота
subscribers = bot.get_updates()
# Отправляем сообщение себе
test = 'Hello, me!'
bot.send_message(chat_id=your_chat_id, text=test )
# Отправляем сообщение каждому подписчику
for subscriber in subscribers:
    user_id = subscriber.message.chat.id
    bot.send_message(chat_id=user_id, text=test)

In [ ]:
# как узнать свой чат ай ди
bot = telebot.TeleBot(token)
# Получаем список обновлений
updates = bot.get_updates()
# Получаем последнее обновление
last_update = updates[-1]
# Получаем chat_id из последнего обновления
chat_id = last_update.message.chat.id
print(chat_id)

In [ ]:
#разговорный бот
import telebot
from telebot import types

bot = telebot.TeleBot(token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start_message(message):
    keyboard = types.InlineKeyboardMarkup()
    url_button = types.InlineKeyboardButton(text="Перейти на сайт", url="https://google.com")
    keyboard.add(url_button)
    bot.send_message(message.chat.id, "Привет! Я тестовый бот. Как дела?", reply_markup=keyboard)

# Обработчик текстового сообщения
@bot.message_handler(content_types=['text'])
def send_text(message):
    if 'как дела' in message.text.lower():
        bot.send_message(message.chat.id, 'Отлично, а у тебя?')
    elif 'привет' in message.text.lower():
        bot.send_message(message.chat.id, 'Привет, как твои дела?')
    else:
        bot.send_message(message.chat.id, 'Я не понимаю, что ты говоришь')

# !!!!! Вместо bot.command_handler используйте bot.message_handler(commands=['...']) !!!!!

# Запуск бота
bot.polling()

In [ ]:
# тест шидулура
# Создаем функцию, которая будет вызываться для отправки документа
def send_document():
    # Создаем тестовые данные
    df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': ['apples', 'oranges', 'bananas']})

    # Сериализуем датафрейм в строку
    df_csv = df.to_csv(index=False)

    # Кодируем строку в байты
    df_bytes = io.BytesIO(df_csv.encode())

    # Отправляем файл
    bot.send_document(chat_id=your_chat_id, document=df_bytes, caption='my_dataframe.csv')

# Функция для обработки команды /start и /help
@bot.message_handler(commands=['start', 'help'])
def handle_start_help(message):
    bot.reply_to(message, "Привет! Чем тебе помочь?")

# Функция для обработки команды /hello
@bot.message_handler(commands=['hello'])
def handle_hello(message):
    t = "Good day to you too!"
    bot.send_message(chat_id=message.chat.id, text=t)

# Функция для обработки команды /nikita
@bot.message_handler(commands=['nikita'])
def handle_nikita(message):
    t1 = "Тебе уже ничем не помочь..."
    bot.send_message(chat_id=message.chat.id, text=t1)

# Функция для обработки команды /inf
@bot.message_handler(commands=['inf'])
def handle_inf(message):
    # Добавьте ваше сообщение
    t2 = "Some info here"
    bot.send_message(chat_id=message.chat.id, text=t2)

# Функция для обработки команды /plot
@bot.message_handler(commands=['plot'])
def handle_plot(message):
    # Создаем данные для графика
    x = np.arange(1, 10, 1)
    y = np.random.choice(5, len(x))

    # Создаем график
    sns.lineplot(x, y)
    plt.title('test')

    # Сохраняем график в объект io.BytesIO
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)

    # Отправляем график через бота
    bot.send_photo(chat_id=message.chat.id, photo=plot_object)

# Функция для обработки команды /df
@bot.message_handler(commands=['df'])
def handle_df(message):
    # Создаем тестовые данные
    df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': ['apples', 'oranges', 'bananas']})

    # Сериализуем датафрейм в строку
    df_csv = df.to_csv(index=False)

    # Кодируем строку в байты
    df_bytes = io.BytesIO(df_csv.encode())

    # Отправляем файл
    bot.send_document(chat_id=message.chat.id, document=df_bytes, caption='my_dataframe.csv')

# Функция для отправки файла ежедневно в 12:00
def send_file_daily():
    schedule.every().day.at("17:10").do(send_document)
    while True:
        schedule.run_pending()
        time.sleep(1)

# Запуск процесса отправки файла ежедневно в 12:00
send_file_daily()

# Запускаем процесс диспетчеризации
bot.polling()

In [6]:
# тест верс 1
# Добавляем обработчик всех входящих сообщений
@bot.message_handler(func=lambda message: message.text is not None)
def handle_text_messages(message):
    # Обрабатываем команду /start и /help
    if message.text.lower() in ['/start', '/help']:
        bot.reply_to(message, "Привет! Чем тебе помочь?")

    # Обрабатываем команду /hello
    elif message.text.lower() == '/hello':
        t = "Good day to you too!"
        bot.send_message(chat_id=message.chat.id, text=t)
    
    # Обрабатываем команду /nikita
    elif message.text.lower() in ['/nikita', 'никита', 'Никита', 'Nikita']:
        t1 = "Тебе уже ничем не помочь..."
        bot.send_message(chat_id=message.chat.id, text=t1)
    
    # Обрабатываем команду /inf
    elif message.text.lower() == '/inf':
        # Добавьте ваше сообщение
        t2 = "Some info here"
        bot.send_message(chat_id=message.chat.id, text=t2)

    # Обрабатываем команду /plot
    elif message.text.lower() == '/plot':
        # Создаем данные для графика
        x = np.arange(1, 10, 1)
        y = np.random.choice(5, len(x))

        # Создаем график
        sns.lineplot(x, y)
        plt.title('test')

        # Сохраняем график в объект io.BytesIO
        plot_object = io.BytesIO()
        plt.savefig(plot_object)
        plot_object.seek(0)

        # Отправляем график через бота
        bot.send_photo(chat_id=message.chat.id, photo=plot_object)

    # Обрабатываем команду /df
    elif message.text.lower() == '/df':
        # Создаем тестовые данные
        df = pd.DataFrame({'Column1': [1, 2, 3], 'Column2': ['apples', 'oranges', 'bananas']})

        # Сериализуем датафрейм в строку
        df_csv = df.to_csv(index=False)

        # Кодируем строку в байты
        df_bytes = io.BytesIO(df_csv.encode())

        # Отправляем файл
        bot.send_document(chat_id=message.chat.id, document=df_bytes, caption='my_dataframe.csv')

# Запускаем процесс диспетчеризации
bot.polling()